## Semantic Vision and Olfaction Fusion
### A Semantic-Based Gas Source Localization with a Mobile Robot Combining Vision and Chemical Sensing
1. Gas detection and classification using electronic nose
Probability is assigned to type of gas detected (e.g., gas smell, smoke smell, rotten food smell, garbage smell, pet smell, etc.), and the possible objects that are related to this gas (e.g., pet bed, bin, heater, dishwasher, etc.).

2. Object detection and recognition using YOLOv3
RGB-D camera is used in this work, where the light intensities are passed to YOLOv3, and the depth information, along with the robot position and heading is used to map the determine object coordinate.

3. Ontologies and semantic queries
Like Appliance category of objects can have gas, rotten food or smoke smells, are located in kitchen, etc.

4. Fusing information with Bayesian Probabilistic Framework
After gas detection, candidate probability is calculated as a product of three conditional probability: (i) the likelihood of of object to belong in a category (a detected object being dishwasher) - conditioned on object detector, (ii) probability of a detected oven to emit smoke or rotten smell - conditioned on prior knowledge, (iii) probability of a detected gas being of a class (a detected gas being smoke) - conditioned on electronic nose.

5. Search
The system continuously updates the 3 probabilities. Upon being triggered of a gas source of significance (e.g., smoke), the system lists probable objects in that can emit the gas. A number of path planning techniques are tested for localizing the odor sources, including, greedy method, MDP method, etc.

Experiment:
experiments were ran in both simulated and real environment.

### Multimodal and Multilingual Understanding of Smells using VilBERT and mUNITER


### Olfaction, Vision, and Semantics for Mobile Robots. Results of the IRO Project

## Indoor Simulator Environments and its Applications with LLM


## Other Multi-modal Fusion Algorithms
### Kimera: an Open-Source Library for Real-Time Metric-Semantic Localization and Mapping

### ConceptFusion: Open-set Multimodal 3D Mapping (https://arxiv.org/pdf/2302.07241)
Objective: fusig pixel-alighed open-set features from foundation models into 3D map via SLAM that are (i) open-set (classifying data to various concepts and levels of details) and (ii) multimodal (language, image, audio, 3D geometry) using LLM and are zero-shot queryable by text, image, audio and quick queries.
Tested on simulated and real-world datasets, tabletop manipulation, autonomous driving platform.

Given sequence of observations $\mathcal{I}$ an open-set multimodal 3D map $\mathcal{M}$ is built. A foundation model $\mathcal{F}_{mode}$ is used as modality-specific encoder to convert query concept of specific mode (text, audio, image...) to query vector $q_{mode}$.

- Fusing pixel-aligned foundation features to 3D
Map representation: 3D map $\mathcal{M}$ is an unordered set of points, each with a vertex position, normal vector, confidence count, color vector, concept vector.

### ConceptGraphs: Open-Vocabulary 3D Scene Graphs for Perception